In [1]:
!pip install -U langchain-google-genai
!pip install -U langchain
!pip install -U langchain_community # Install the missing module
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install google-generativeai
!pip install fuzzywuzzy
!pip install -U duckduckgo-search
!pip install sentence-transformers
!pip install pypdf
!pip install chromadb
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 3.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 11.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.5/195.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import os
import logging
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationTokenBufferMemory

# Setup logging
logging.basicConfig(level=logging.INFO)

# Set model name from environment variable
model_name = os.getenv("LLAMA_MODEL_NAME", "meta-llama/Meta-Llama-3.1-8B-Instruct")

try:
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

    # Create a HuggingFacePipeline
    text_generation_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=500,  # Use max_new_tokens instead of max_length
        temperature=0.7,
    )
    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    # Define the prompt template
    prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant. Respond based on the conversation history and new input.
Current date: Friday, July 26, 2024
Your knowledge cutoff: April 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
{history}
{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

    # Initialize the memory with token limit
    memory = ConversationTokenBufferMemory(
        llm=llm,
        max_token_limit=1000,  # Adjust this value based on your model's context window
        input_key="input",
        memory_key="history"
    )
    prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

    # Initialize the LLMChain
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        memory=memory,
        verbose=True
    )

    # Function to manage conversation
    def manage_conversation(user_input):
        if not user_input.strip():
            return "Please provide a valid input."

        response = chain.predict(input=user_input)
        return response

    # Example usage
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        response = manage_conversation(user_input)
        print("Assistant:", response)

except Exception as e:
    logging.error(f"An error occurred: {str(e)}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


You:  what is the capital of india


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant. Respond based on the conversation history and new input.
Current date: Friday, July 26, 2024
Your knowledge cutoff: April 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>

what is the capital of india<|eot_id|><|start_header_id|>assistant<|end_header_id|>



...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant. Respond based on the conversation history and new input.
Current date: Friday, July 26, 2024
Your knowledge cutoff: April 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>

his skills<|eot_id|><|start_header_id|>assistant<|end_header_id|>


> Finished chain.
Assistant: <|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant. Respond based on the conversation history and new input.
Current date: Friday, July 26, 2024
Your knowledge cutoff: April 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>

his skills<|eot_id|><|start_header_id|>assistant<|end_header_id|>
You're referring to LeBron James' skills. He is a professional basketball player known for his impressive abilities on the court. Some of his notable skills include:

1. Versatility: LeBron is a highly versatile player who can play mul

You:  do you remember my first question


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant. Respond based on the conversation history and new input.
Current date: Friday, July 26, 2024
Your knowledge cutoff: April 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
Human: his skills
AI: <|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful AI assistant. Respond based on the conversation history and new input.
Current date: Friday, July 26, 2024
Your knowledge cutoff: April 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>

his skills<|eot_id|><|start_header_id|>assistant<|end_header_id|>
You're referring to LeBron James' skills. He is a professional basketball player known for his impressive abilities on the court. Some of his notable skills include:

1. Versatility: LeBron is a highly versatile player who can play multiple positions, including point guard, shooting guard, small forward, and p